In [3]:
import xarray as xr
import numpy as np
import pickle
import networkx as nx
import torch
# chunks = {'time': 10000}
# temperature_2m = xr.open_mfdataset('./data/all_5.625deg/2m_temperature/2m_temperature_*.nc', chunks=chunks)
# u_wind_10m = xr.open_mfdataset('./data/all_5.625deg/10m_u_component_of_wind/10m_u_component_of_wind_*.nc', chunks=chunks)
# v_wind_10m = xr.open_mfdataset('./data/all_5.625deg/10m_v_component_of_wind/10m_v_component_of_wind_*.nc', chunks=chunks)
geopotential = xr.open_mfdataset('./data/all_5.625deg/geopotential/geopotential_*.nc')
# geopotential_500 = xr.open_mfdataset('./data/all_5.625deg/geopotential_500/geopotential_*.nc', chunks=chunks) #single level, vs 13 levels
vorticity_potential = xr.open_mfdataset('./data/all_5.625deg/potential_vorticity/potential_vorticity_*.nc')
vorticity = xr.open_mfdataset('./data/all_5.625deg/vorticity/vorticity_*.nc')
# relative_humidity = xr.open_mfdataset('./data/all_5.625deg/relative_humidity/relative_humidity_*.nc')
specific_humidity = xr.open_mfdataset('./data/all_5.625deg/specific_humidity/specific_humidity_*.nc')
temperature = xr.open_mfdataset('./data/all_5.625deg/temperature/temperature_*.nc')
# temperature_850 = xr.open_mfdataset('./data/all_5.625deg/temperature_850/temperature_850hPa_*.nc') #single level, vs 13 levels
solar_radiation = xr.open_mfdataset('./data/all_5.625deg/toa_incident_solar_radiation/toa_incident_solar_radiation_*.nc')
# cloud_cover = xr.open_mfdataset('./data/all_5.625deg/total_cloud_cover/total_cloud_cover_*.nc')
precipitation_total = xr.open_mfdataset('./data/all_5.625deg/total_precipitation/total_precipitation_*.nc')
u_wind = xr.open_mfdataset('./data/all_5.625deg/u_component_of_wind/u_component_of_wind_*.nc')
v_wind = xr.open_mfdataset('./data/all_5.625deg/v_component_of_wind/v_component_of_wind_*.nc')

data = xr.merge([
    # temperature_2m,
    # u_wind_10m,
    # v_wind_10m,
    geopotential,
    # geopotential_500,
    vorticity_potential,
    vorticity,
    # relative_humidity,
    specific_humidity,
    temperature,
    # temperature_850,
    solar_radiation,
    # cloud_cover,
    precipitation_total,
    u_wind,
    v_wind
    ])

data

,Array,Chunk
Bytes,34.78 GiB,892.12 MiB
Shape,"(350640, 13, 32, 64)","(8784, 13, 32, 64)"
Count,120 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,34.78 GiB,892.12 MiB
Shape,"(350640, 13, 32, 64)","(8784, 13, 32, 64)"
Count,120 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [5]:
grouped = data.stack(node=['lat', 'lon'])

levels = [50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000]
var_dict = {
    'z': levels,
    'pv': levels,
    'vo': levels,
    'q': levels,
    't': levels,
    'u': levels,
    'v': levels,
    'tisr': None,
    'tp': None, 
}
generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])

data_list = []
def process_levels(d, data_ref):
    _data_list = []    
    for var, levels in d.items():
        print(var, levels)
        try:
            if levels is None:
                _data_list.append(data_ref[var].expand_dims({'level': generic_level}, 1))
            else:    
                _data_list.append(data_ref[var].sel(level=levels))
        except ValueError:
            print('value error', var)
            _data_list.append(data_ref[var].expand_dims({'level': generic_level}, 1))
        except KeyError:
            _data_list.append(data_ref[var])
            
    return _data_list
data_list = process_levels(var_dict, grouped) 
_data = xr.concat(data_list, 'level').transpose('time', 'node', 'level')
_data

z [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
pv [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
vo [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
q [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
t [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
u [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
v [50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000]
tisr None
tp None


/home/arturas/anaconda3/lib/python3.10/site-packages/xarray/core/variable.py:1786: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._stack_once(dims, new_dim)


,Array,Chunk
Bytes,248.79 GiB,892.12 MiB
Shape,"(350640, 2048, 93)","(8784, 2048, 13)"
Count,2684 Tasks,360 Chunks
Type,float32,numpy.ndarray


In [4]:
from torch_geometric.utils.convert import from_networkx
sphere_graph = pickle.load(open('./sphere_graph.pickle', 'rb'))
nodes = [(u, v) for u, v in sphere_graph]

graph_data = from_networkx(sphere_graph)
edge_index = graph_data.edge_index
edge_weight = graph_data.distance

edge_index, edge_weight

(tensor([[   0,    0,    0,  ..., 2047, 2047, 2047],
         [   0,    1,   63,  ..., 2045, 2046, 2047]]),
 tensor([    0.0000, 30815.5723, 30815.5723,  ..., 61557.0859, 30815.5723,
             0.0000]))

In [15]:
t = torch.Tensor(_data[:100].load().data)

In [8]:
_data.shape[-1]

93

In [16]:
t.shape

torch.Size([100, 2048, 93])

In [111]:
from tsl.data import SpatioTemporalDataset

torch_dataset = SpatioTemporalDataset(target=_data,
                                      connectivity=(edge_index, edge_weight),
                                      horizon=12,
                                      window=12,
                                      stride=1)

2023-10-22 14:26:26.750680: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-22 14:26:26.771520: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-22 14:26:26.771542: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-22 14:26:26.771558: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-22 14:26:26.775825: I tensorflow/core/platform/cpu_feature_g

In [97]:
sq = [n.item() for n in _data.node]

In [98]:
np.array_equal(nodes, sq)

True

In [9]:
_data[:10, 0, :]

<xarray.DataArray 'z' (time: 10, level: 93)>
dask.array<getitem, shape=(10, 93), dtype=float32, chunksize=(10, 13), chunktype=numpy.ndarray>
Coordinates:
  * level    (level) int64 50 100 150 200 250 300 400 ... 700 850 925 1000 1 1
  * time     (time) datetime64[ns] 1979-01-01 ... 1979-01-01T09:00:00
    node     object (-87.1875, 0.0)
    lat      float64 -87.19
    lon      float64 0.0
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [8]:
df = _data.to_dask_dataframe()
df

/home/arturas/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/arturas/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3460: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid crea

,level,time,node,lat,lon,z
npartitions=93,,,,,,
0,int64,datetime64[ns],object,float64,float64,float32
718110720,...,...,...,...,...,...
...,...,...,...,...,...,...
66066186240,...,...,...,...,...,...
66784296959,...,...,...,...,...,...


In [18]:
df.node[(-87.1875, 0.0)]

IndexingError: Too many indexers

In [30]:
import pandas as pd
np.random.seed(123)
times = pd.date_range("2000-01-01", "2001-12-31", name="time")

annual_cycle = np.sin(2 * np.pi * (times.dayofyear.values / 365.25 - 0.28))
base = 10 + 15 * annual_cycle.reshape(-1, 1)
tmin_values = base + 3 * np.random.randn(annual_cycle.size, 3)
tmax_values = base + 10 + 3 * np.random.randn(annual_cycle.size, 3)

#Example dataset from xarray examples
ds = xr.Dataset(
    {
        "tmin": (("time", "location"), tmin_values),
        "tmax": (("time", "location"), tmax_values),
    },
    {"time": times, "location": ["IA", "IN", "IL"]},
)
ds

<xarray.Dataset>
Dimensions:   (time: 731, location: 3)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2001-12-31
  * location  (location) <U2 'IA' 'IN' 'IL'
Data variables:
    tmin      (time, location) float64 -8.037 -1.788 -3.932 ... -1.346 -4.544
    tmax      (time, location) float64 12.98 3.31 6.779 ... 6.636 3.343 3.805

In [31]:
da_nans=ds.where(ds.tmin>0)
da_nans

<xarray.Dataset>
Dimensions:   (time: 731, location: 3)
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2001-12-31
  * location  (location) <U2 'IA' 'IN' 'IL'
Data variables:
    tmin      (time, location) float64 nan nan nan nan nan ... nan 2.063 nan nan
    tmax      (time, location) float64 nan nan nan nan nan ... nan 6.636 nan nan

In [35]:
da_stacked=da_nans.stack(dim=['time','location'])


dask.array<values, shape=(nan,), dtype=object, chunksize=(nan,), chunktype=numpy.ndarray>

In [ ]:
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric.data import Data
from torch_geometric.utils.convert import from_networkx

sphere_graph = pickle.load(open('./sphere_graph.pickle', 'rb'))
class WeatherDataLoader(StaticGraphTemporalSignal):
    def __init__(self, data, lags: int = 112, target_length: int  = 40, temporal_resolution: int = 3) -> None:
        """_summary_

        Args:
            data (xarray handle):
                data reference for weatherbench
            lags (int, optional): 
                Input timesteps. Defaults to 112.
            target_length (int, optional): 
                Output timestep length. Defaults to 40.
            temporal_resolution (int, optional): 
                Defines what timesteps to take for both input and output. 1 - every timestep (hour), 3 - every 3rd hour. Defaults to 3.
        """
        super(WeatherDataLoader).__init__()
        # data - xarray reference
        self._get_edges()
        self._get_nodes()
        
        # backfill nan's across time dimension
        # self.data = data.bfill(dim="time")
        graph_data = from_networkx(sphere_graph)
        self.edge_index = graph_data.edge_index
        self.edge_weight = graph_data.distance
        self.data = data.stack(node=['lat', 'lon'])
        
        self.lags = lags
        self.target_length = target_length
        self.temporal_resolution = temporal_resolution
        self.tmp_resolution_adjusted_indices = None
        self._set_snapshot_count()
        self.levels = [50,  100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000]
        self.input_dict = {
            'z': self.levels,
            'pv': self.levels,
            'vo': self.levels,
            'q': self.levels,
            't': self.levels,
            'u': self.levels,
            'v': self.levels,
            'tisr': None,
            'tp': None, 
        }
        self.output_dict = {
            'z': [500],
            't': [850],
        }
        self.generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
        # self.data_list = self._get_input_data_list()
        # self._data = self.get_stacked_data(self.data_list)
        # self._output = self.get_stacked_data(self._get_output_data_list())
        self.edge_coords = None
        self.input_data_view = self._produce_list(self.input_dict, self.data)
        self.target_data_view = self._produce_list(self.output_dict, self.data)
        
        
    def _get_input_data_list(self):
        # data_list = []
        # for var, levels in self.var_dict.items():
        #     print(var, levels)
        #     try:
        #         if levels is None:
        #             data_list.append(self.data[var].expand_dims({'level': self.generic_level}, 1))
        #         else:    
        #             data_list.append(self.data[var].sel(level=levels))
        #     except ValueError:
        #         print('value error', var)
        #         data_list.append(self.data[var].expand_dims({'level': self.generic_level}, 1))
        #     except KeyError:
        #         data_list.append(self.data[var])
                
        # return data_list
        return self._produce_list(self.var_dict)
    
    def _get_output_data_list(self):
        output_dict = {
            'z': [500],
            't': [850],
        }
        return self._produce_list(output_dict)
    
    def _produce_list(self, var_dict, data):
        data_list = []
        for var, levels in var_dict.items():
            print(var, levels)
            try:
                if levels is None:
                    data_list.append(data[var].expand_dims({'level': self.generic_level}, 1))
                else:    
                    data_list.append(data[var].sel(level=levels))
            except ValueError:
                print('value error', var)
                data_list.append(data[var].expand_dims({'level': self.generic_level}, 1))
            except KeyError:
                data_list.append(data[var])
                
        return self.get_stacked_data(data_list)

    def get_stacked_data(self, data_list):
        return xr.concat(data_list, 'level').transpose('time', 'node', 'level')
        
    def _set_snapshot_count(self):
        window_size = self._get_time_slice(0)
        self.snapshot_count = len(self.data.time) // len(window_size)
    
    def _get_window_indices(self):
        if self.tmp_resolution_adjusted_indices is not None:
            return self.tmp_resolution_adjusted_indices
        
        time_window = np.arange(self.lags + self.target_length)
        self.tmp_resolution_adjusted_indices = time_window * self.temporal_resolution
        
        return self.tmp_resolution_adjusted_indices
    
    def _get_time_slice(self, window_index):
        window = self._get_window_indices()
        window_to_get = window + (len(window) * self.temporal_resolution * window_index)
        return window_to_get
        
        
    def _load_slice(self, idx: int):
        window = self._get_time_slice(idx)
        indices = self._get_time_slice(idx)[:self.lags]
        target_indices = self._get_time_slice(idx)[self.lags:]
        # timeframe = self._load_timeframe(indices, self._data)
        # target_frame = self._load_timeframe(target_indices, self._output)
        # timeframe = self._load_timeframe(window).bfill(dim="time")
        input_frame = self._load_timeframe(indices, self.input_data_view)
        target_frame = self._load_timeframe(target_indices, self.target_data_view)
        
        # input_frame = timeframe.isel(time=np.arange(self.lags))
        # input_frame = self._produce_list(self.input_dict, input_frame)
        
        # target_frame = timeframe.isel(time=np.arange(self.lags + self.target_length)[self.lags:])
        # target_frame = self._produce_list(self.output_dict, target_frame)
        
        return torch.Tensor(input_frame.load().data), torch.Tensor(target_frame.load().data)
    
    def _get_graph_data(self, data):
        # attrs_over_time = []
        # for timestep in data:
        #     attrs = {}
        #     for node in self.nodes:
        #         d = timestep.sel(lat=node[0], lon=node[1], method='nearest').values
        #         attrs[node] = {'data': d}
        #     attrs_over_time.append(attrs)
            
        # # timesteps_as_graphs = []
        # timesteps_as_data_objects = []
        # for attr in attrs_over_time:
        #     cp = sphere_graph.copy()
        #     nx.set_node_attributes(cp, attr)
        #     # timesteps_as_graphs.append(cp)
        #     g = from_networkx(cp, group_edge_attrs=['distance'], group_node_attrs=['data'])
        #     timesteps_as_data_objects.append(g)
            
        # data_sequence_x = torch.zeros((*timesteps_as_data_objects[0].x.T.shape, len(timesteps_as_data_objects)))

        # for i, time_data_object in enumerate(timesteps_as_data_objects):
        #     data_sequence_x[:, :, i] = time_data_object.x.T
            
        # data_sequence_x.dtype

        # common_data = timesteps_as_data_objects[0]
        
        # return Data(x=data_sequence_x, edge_index=common_data.edge_index, edge_attr=common_data.edge_attr.reshape(-1))
        timesteps_as_data_objects = {}
        for node in self.nodes:
            data_slice = data.sel(lat=node[0], lon=node[1], method='nearest').values
            timesteps_as_data_objects[node]= {'data': data_slice.T}
        
        cp = sphere_graph.copy()
        nx.set_node_attributes(cp, timesteps_as_data_objects)
        g = from_networkx(cp, group_edge_attrs=['distance'], group_node_attrs=['data'])
        
        nodes, values, timesteps = g.x.shape
        
        stacked = []
        for t in range(timesteps):
            frame = g.x[:,:, t]
            stacked.append(frame.T)
        stacked = torch.stack(stacked, dim=2)
        
        g.x = stacked.T
        return g
    
    def _load_timeframe(self, timeframe, data):
        return data.isel(time=timeframe).bfill(dim="time")
        # return self._produce_list(var_dict, data)
    
    def _get_graphs(self, inputs, targets):
        input_data = self._get_graph_data(inputs)
        output_data = self._get_graph_data(targets)
        
        snapshot = Data(
            x=input_data.x,
            edge_index=input_data.edge_index,
            edge_attr=input_data.edge_attr,
            y=output_data.x)
        
        return snapshot
    
    def __getitem__(self, index):
        inputs, targets = self._load_slice(index)
        return inputs, targets
        # snapshot = self._get_graphs(inputs, targets)
        # return snapshot
        
    
    def _get_edges(self):
        self.edge_coords = sphere_graph.edges() 
        self.edges = sphere_graph.edges()
    
    def _get_nodes(self): 
        self.nodes = [(u, v) for u, v in sphere_graph]
        
    def _get_edges_and_weights(self):
        pass
    
    def __len__(self):
        return self.snapshot_count
    
    
    # def _load_timeframe(self, timeframe, data):
    #     # load timeframe, slice into specific nodes afterwards
    #     return data.isel(time=timeframe).load()